In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install pyspark

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-10-16 18:30:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.58MB/s    in 0.2s    

2020-10-16 18:30:49 (3.58 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

# Show DataFrame
video_game_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-01-31 00:08:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

# Count

In [ ]:
# Count the total rows
video_game_df.count()


1785997

In [ ]:
# Drop null values and recount the rows
video_game_df = video_game_df.dropna()
video_game_df.count()

1785886

## Schema

In [ ]:
# Print schema to confirm the correct data types
video_game_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



# Transform

In [ ]:
# Create review dataframe to match "review_id_table" table
video_game_review_id_df = video_game_df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
video_game_review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809|2015-01-31 00:08:00|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175|2015-01-31 00:08:00|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668|2015-01-31 00:08:00|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350|2015-01-31 00:08:00|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511|2015-01-31 00:08:00|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615|2015-01-31 00:08:00|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603|2015-01-31 00:08:00|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517|2015-01-31 00:08:00|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890|2015-01-31 00:08:00|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651|2015-01-31 00:08:00|
|R1VTIA3JTYBY02|     1065

In [ ]:
# Create product dataframe to match "products" table
video_game_products_df = video_game_df.select(["product_id","product_title"])
video_game_products_df = video_game_products_df.dropDuplicates()
video_game_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create customers dataframe to match "customers" table
from pyspark.sql.functions import desc


video_game_customers_df = video_game_df.groupby("customer_id").agg({"customer_id":"count"})
video_game_customers_df = video_game_customers_df.orderBy(desc("count(customer_id)"))
video_game_customers_df = video_game_customers_df.withColumnRenamed("count(customer_id)", "customer_count")
video_game_customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52759271|           909|
|   42418272|           856|
|   45838740|           839|
|   24846840|           614|
|   24594001|           460|
|   40955995|           459|
|   48278407|           439|
|   27364030|           361|
|   30678701|           326|
|   50648042|           301|
|   14539589|           288|
|   10541465|           271|
|   38752775|           263|
|   43092063|           257|
|   45261621|           235|
|   44603910|           233|
|   14886512|           233|
|   37687227|           226|
|   10075230|           218|
|   24476167|           214|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create vine dataframe to match "vine_table" table
from pyspark.sql.functions import col

video_game_vine_table_df = video_game_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
video_game_vine_table_df = video_game_vine_table_df.filter(col("vine")  == "Y")
video_game_vine_table_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RX3CK2RLSAF7T|          5|            0|          0|   Y|
|R253CC14DRG5WQ|          3|            0|          0|   Y|
| R1ZXWP3JKVXMU|          5|            0|          0|   Y|
|R1I01YYQ35M80G|          5|            0|          0|   Y|
| RWKQQPG100CQ4|          4|            2|          4|   Y|
|R1R0J8S1VGALUY|          1|            1|          2|   Y|
|R28165EG0N9CSN|          4|            1|          2|   Y|
|R1CH0ZXGIMVEFC|          5|            1|          2|   Y|
|R2BFEJHEKH4YWZ|          5|            3|          4|   Y|
| R73CGNRBSWLLM|          5|            0|          0|   Y|
| RYIG0BYEZP7WM|          2|            1|          1|   Y|
| RHGGJU4WY3YBL|          2|            0|          3|   Y|
| RUV0RB5H9ENT5|          4|            0|          0|   Y|
|R20AUQF3VKFU4I|          4|            

## **Load into RDS (Postgres)**

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bigdatach3act3db.cw1hhyrwelau.us-east-2.rds.amazonaws.com:5432/amazongamereviews"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id_df table in RDS

video_game_review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS

video_game_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

video_game_customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS

video_game_vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)